In [1]:
import sys
import torch
import os
import math
from os import makedirs
import numpy as np
import open3d as o3d
from random import randint
from scipy.spatial.transform import Rotation
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import random
import open3d as o3d

np.set_printoptions(suppress=True)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [ ]:
rot = qvec2rotmat([1.13133e-08,0.965926,0.258819,-4.2222e-08])
rot2 = Rotation.from_quat([1.13133e-08,0.965926,0.258819,-4.2222e-08]).as_matrix()

print(rot)
print('')
print(rot2)

In [2]:
# Visualize the final point cloud
ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\ScanNet\\scene0000_00\\sens_read\\ply"
first_ply_file_path = os.path.join(ply_path, '0.ply')
Global_ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting"
ply_file_path = os.path.join(Global_ply_path, 'Global.ply')

# 4X4 transformation matrix
transformation_matrix =  [[1, 0, 0, 0.5],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]


ply_files = os.listdir(ply_path)
pcd_list = []
# for file in ply_files[:1]:
#     if file.endswith(".ply"):
#         pcd_file = os.path.join(ply_path, file)
#         pcd = o3d.io.read_point_cloud(pcd_file)
#         pcd.transform(transformation_matrix)
#         pcd_list.append(pcd)

pcd_list.append(o3d.io.read_point_cloud(ply_file_path))
# pcd_list.append(o3d.io.read_point_cloud(first_ply_file_path))

# Visualize the final point cloud
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
o3d.visualization.draw_geometries(pcd_list)


[Open3D WARNING] Read PLY failed: unable to open file: G:\Universitat Siegen\SA\P-GPU\Code\gaussian-splatting\Global.ply


In [ ]:
# Visualize the filtered point cloud
filtered_ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply_filtered" 

ply_files = os.listdir(filtered_ply_path)
pcd_list = []
for file in ply_files:
    if file.endswith(".ply"):
        pcd_file = os.path.join(filtered_ply_path, file)
        pcd = o3d.io.read_point_cloud(pcd_file)
        pcd.transform(transformation_matrix)
        pcd_list.append(pcd)

# Visualize the final point cloud
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
o3d.visualization.draw_geometries(pcd_list)


In [ ]:
a = torch.Tensor([[1,2],[4,5]])
b = torch.Tensor([[3,4],[4,5],[0,9]])

mask = torch.isin(a,b)

mask_all_true = mask.all(dim=1)

mask_all_true